# Ocean Color

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Time period
start = "1997-09"
stop = "1998-08"

## Define request

In [ ]:
collection_id = "satellite-ocean-colour"

request = {
    "variable": "mass_concentration_of_chlorophyll_a",
    "projection": "regular_latitude_longitude_grid",
    "version": "6_0",
    "format": "zip",
}


requests = download.update_request_date(
    request, start=start, stop=stop, stringify_dates=True
)

## Functions to cache

In [ ]:
def regionalised_spatial_weighted_mean(ds, lon_slice, lat_slice):
    da = ds["chlor_a"]
    da = utils.regionalise(da, lon_slice=lon_slice, lat_slice=lat_slice)
    da = da.where((da > 1.0e-3) & (da < 1.0e2))
    with xr.set_options(keep_attrs=True):
        da = 10 ** diagnostics.spatial_weighted_mean(np.log10(da))
    da.attrs["long_name"] = da.attrs["long_name"].replace(" (not log-transformed)", "")
    return da.to_dataset()

## Download and transform data

In [ ]:
lon_slice = slice(-180, 180)
datasets = []
for lats in zip(range(90, -90, -30), range(60, -120, -30)):
    print(f"{lats=}")
    ds = download.download_and_transform(
        collection_id,
        requests,
        transform_func=regionalised_spatial_weighted_mean,
        transform_func_kwargs={"lon_slice": lon_slice, "lat_slice": slice(*lats)},
        chunks={"year": 1, "month": 1},
    )
    datasets.append(ds.expand_dims(latitudes=[str(lats)]))
ds = xr.concat(datasets, "latitudes")
da_daily = ds["chlor_a"]
da_monthly = da_daily.resample(time="M").mean()
da_seasonality = da_daily.groupby("time.month").mean()

## Daily timeseries

In [ ]:
window = 60
for i, (lats, da) in enumerate(da_daily.groupby("latitudes")):
    da.plot(color=f"C{i}", ls="-", label=f"{lats}: daily")
    da.rolling(time=window, min_periods=1).mean().plot(
        color=f"C{i}", ls="--", label=f"{lats}: {window}-day running"
    )
plt.grid()
_ = plt.legend(bbox_to_anchor=(1, 1))

## Monthly timeseries

In [ ]:
window = 6
for i, (lats, da) in enumerate(da_monthly.groupby("latitudes")):
    da.plot(color=f"C{i}", ls="-", label=f"{lats}: monthly")
    da.rolling(time=window, min_periods=1).mean().plot(
        color=f"C{i}", ls="--", label=f"{lats}: {window}-month running"
    )
plt.grid()
_ = plt.legend(bbox_to_anchor=(1, 1))

## Seasonality

In [ ]:
window = 6
for i, (lats, da) in enumerate(da_seasonality.groupby("latitudes")):
    da.plot(color=f"C{i}", ls="-", label=f"{lats}: seasonality", marker="o")
    da.rolling(month=window, min_periods=1).mean().plot(
        color=f"C{i}", ls="--", label=f"{lats}: {window}-month running", marker="o"
    )
plt.title(f"From {start} to {stop}")
plt.grid()
_ = plt.legend(bbox_to_anchor=(1, 1))